# Исследование данных об инвестициях венчурных фондов в компании-стартапы

1. Посчитайте, сколько компаний закрылось.

    SELECT COUNT(name)  
    FROM company  
    WHERE status = 'closed';

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

    SELECT SUM(funding_total)  
    FROM company  
    WHERE category_code = 'news'  
    AND country_code = 'USA'  
    GROUP BY name  
    ORDER BY SUM(funding_total) DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

    SELECT SUM(price_amount)  
    FROM acquisition  
    WHERE term_code = 'cash'  
    AND EXTRACT(year FROM acquired_at) IN (2011, 2012, 2013);

4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

    SELECT first_name,  
           last_name,  
           twitter_username  
    FROM people  
    WHERE twitter_username LIKE 'Silver%';  

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

    SELECT id,
           first_name,
           last_name,
           company_id,
           twitter_username,
           created_at,
           updated_at
    FROM people
    WHERE twitter_username LIKE '%money%'
    AND last_name LIKE 'K%';

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

    SELECT country_code,
           SUM(funding_total) AS s
    FROM company
    GROUP BY country_code
    ORDER BY s DESC;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

    SELECT funded_at,
           MIN(raised_amount),
           MAX(raised_amount)
    FROM funding_round
    GROUP BY funded_at
    HAVING MIN(raised_amount) != MAX(raised_amount)
    AND MIN(raised_amount) != 0;

8. Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

    SELECT id,
           name,
           founded_at,
           domain,
           twitter_username,
           country_code,
           investment_rounds,
           invested_companies,
           milestones,
           created_at,
           updated_at,
           CASE
               WHEN invested_companies >= 100 THEN 'high_activity'
               WHEN invested_companies >= 20 THEN 'middle_activity'
               ELSE 'low_activity'
           END
    FROM fund;

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

    SELECT ROUND(AVG(investment_rounds), 0) AS a,
           CASE
               WHEN invested_companies>=100 THEN 'high_activity'
               WHEN invested_companies>=20 THEN 'middle_activity'
               ELSE 'low_activity'
           END AS activity
    FROM fund
    GROUP BY activity
    ORDER BY a;

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

    SELECT country_code,
           MIN(invested_companies),
           MAX(invested_companies),
           AVG(invested_companies)
    FROM fund
    WHERE EXTRACT(year FROM founded_at) IN (2010, 2011, 2012)
    GROUP BY country_code
    HAVING MIN(invested_companies) > 0
    ORDER BY AVG(invested_companies) DESC, country_code
    LIMIT 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

    SELECT p.first_name,
           p.last_name,
           e.instituition
    FROM people AS p
    LEFT JOIN education AS e ON p.id=e.person_id;

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

    SELECT c.name,
           COUNT(DISTINCT e.instituition) AS ei
    FROM company AS c
    LEFT JOIN people AS p ON c.id=p.company_id
    INNER JOIN education AS e ON p.id=e.person_id
    GROUP by c.name
    ORDER BY ei DESC
    LIMIT 5;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

    SELECT DISTINCT c.name
    FROM company AS c
    LEFT JOIN funding_round AS fr ON c.id=fr.company_id
    WHERE c.status='closed'
    AND fr.is_first_round=1
    AND fr.is_last_round=1;

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

    SELECT id
    FROM people
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON c.id=fr.company_id
        WHERE c.status='closed'
        AND fr.is_first_round=1
        AND fr.is_last_round=1);

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

    SELECT DISTINCT p.id,
           e.instituition
    FROM people AS p
    INNER JOIN education AS e ON p.id=e.person_id
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON c.id=fr.company_id
        WHERE c.status='closed'
        AND fr.is_first_round=1
        AND fr.is_last_round=1);

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

    SELECT p.id,
           COUNT(e.instituition)
    FROM people AS p
    INNER JOIN education AS e ON p.id=e.person_id
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON c.id=fr.company_id
        WHERE c.status='closed'
        AND fr.is_first_round=1
        AND fr.is_last_round=1)
    GROUP BY p.id;

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

    SELECT SUM(count)/COUNT(id)
    FROM (SELECT p.id,
           COUNT(e.instituition)
    FROM people AS p
    INNER JOIN education AS e ON p.id=e.person_id
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON c.id=fr.company_id
        WHERE c.status='closed'
        AND fr.is_first_round=1
        AND fr.is_last_round=1)
    GROUP BY p.id) AS t1;

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.

    SELECT SUM(count)/COUNT(id)
    FROM (SELECT p.id,
           COUNT(e.instituition)
    FROM people AS p
    INNER JOIN education AS e ON p.id=e.person_id
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON c.id=fr.company_id
        WHERE c.name='Facebook')
    GROUP BY p.id) AS t1;

19. Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

    SELECT f.name AS name_of_fund,
           c.name AS name_of_company,
           fr.raised_amount AS amount
    FROM investment AS i
    LEFT JOIN company AS c ON i.company_id=c.id
    LEFT JOIN fund AS f ON i.fund_id=f.id
    LEFT JOIN funding_round AS fr ON i.funding_round_id=fr.id
    WHERE c.milestones>6
    AND EXTRACT(year FROM fr.funded_at) IN (2012, 2013);

20. Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

    WITH
    t1 AS (SELECT a.acquiring_company_id,
               a.price_amount,
               c.name,
               c.funding_total,
               ROUND(a.price_amount/c.funding_total)
        FROM acquisition AS a
        LEFT JOIN company AS c ON a.acquired_company_id=c.id
        WHERE a.price_amount>0
        AND c.funding_total>0
        ORDER BY a.price_amount DESC)
    
    SELECT c.name,
           t1.price_amount,
           t1.name,
           t1.funding_total,
           t1.ROUND
    FROM t1 LEFT JOIN company AS c ON t1.acquiring_company_id=c.id
    ORDER BY t1.price_amount DESC, t1.name
    LIMIT 10;

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

    SELECT c.name,
           EXTRACT(month FROM fr.funded_at)
    FROM company AS c
    LEFT JOIN funding_round AS fr ON c.id=fr.company_id
    WHERE c.category_code='social'
    AND EXTRACT(year FROM fr.funded_at) IN (2010, 2011, 2012, 2013)
    AND fr.raised_amount>0;

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

    WITH
    t_fr AS (SELECT EXTRACT(month FROM fr.funded_at) AS m,
                    COUNT(DISTINCT f.name) AS number_of_f
             FROM investment AS i
             LEFT JOIN funding_round AS fr ON i.funding_round_id=fr.id
             LEFT JOIN fund AS f ON i.fund_id=f.id
             WHERE f.country_code='USA'
             AND EXTRACT(year FROM fr.funded_at) BETWEEN 2010 AND 2013
             GROUP BY m),
    t_a AS (SELECT EXTRACT(month FROM a.acquired_at) AS m,
                   COUNT(a.acquired_company_id) AS number_of_c,
                   SUM(a.price_amount) AS total_m
            FROM acquisition AS a
            WHERE EXTRACT(year FROM a.acquired_at) BETWEEN 2010 AND 2013
            GROUP BY m)
    SELECT t_fr.m,
           t_fr.number_of_f,
           t_a.number_of_c,
           t_a.total_m
    FROM t_fr INNER JOIN t_a ON t_fr.m=t_a.m;

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

    WITH
    t1 AS (SELECT AVG(funding_total) AS inv_2011,
               country_code
        FROM company
        WHERE EXTRACT(year FROM founded_at)=2011
        GROUP BY country_code),
    t2 AS (SELECT AVG(funding_total) AS inv_2012,
               country_code
        FROM company
        WHERE EXTRACT(year FROM founded_at)=2012
        GROUP BY country_code),
    t3 AS (SELECT AVG(funding_total) AS inv_2013,
               country_code
        FROM company
        WHERE EXTRACT(year FROM founded_at)=2013
        GROUP BY country_code)
    
    SELECT t1.country_code,
           t1.inv_2011,
           t2.inv_2012,
           t3.inv_2013
    FROM t1
    INNER JOIN t2 ON t1.country_code=t2.country_code
    INNER JOIN t3 ON t1.country_code=t3.country_code
    ORDER BY t1.inv_2011 DESC;